In [2]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt 
import matplotlib as mpl
import seaborn as sns 
import geopandas as gpd
import pandas as pd 
import numpy as np
import requests
import shapely
import pickle
import progressbar
import geopy.distance
import geopandas as gpd
import pyproj
import os
import matplotlib.cm as cm
import matplotlib.colors as colors
import itertools
import warnings
import random
import timeit
import time
import threading
import functions
from joblib import Parallel, delayed
from functions import *
from shapely.geometry import Point, Polygon, LineString 
from shapely.ops import nearest_points
from shapely.ops import transform
from shapely import geometry, ops
from functools import partial
from itertools import combinations
from multiprocessing import Pool
from random import choice, randint
from scipy.sparse import lil_matrix
warnings.simplefilter('ignore')

C:\Users\alexa\anaconda3\envs\ox\lib\site-packages\pyproj\__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


In [ ]:
#Read GeoJson package from Vegvesen
ruttger_link_geom = gpd.read_file("Data/kartdata4.geojson")
ruttger_link_geom['length'] = ruttger_link_geom['geometry'].length

In [3]:
#Read OD Nodes and All current ChargingStationsNodes
OD = pd.read_csv("Data/OD_new.csv", encoding="Cp1252")
OD = OD.drop('Unnamed: 0', axis=1)
# OD_test = pd.read_csv("Data/OD_test.csv", encoding="iso8859_10")
# OD_test = OD_test.drop('Unnamed: 0', axis=1)
CS = pd.read_csv("Data/AllCurrentCS.csv", encoding="Cp1252")
CS = CS.drop('Unnamed: 0', axis=1)

In [ ]:
# Transform project into lat-long coordinate system (ESPG:4326)
project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:8687'), # source coordinate system
    pyproj.Proj(init='epsg:4326')) # destination coordinate system

ruttger_link_geom['geometry'] = ruttger_link_geom['geometry'].apply(lambda x: transform(project.transform, x))

In [ ]:
nodes, edges = functions.build_network_data(ruttger_link_geom)


In [ ]:
graph_attrs = {'crs': 'epsg:4326', 'simplified': False}
G = ox.graph_from_gdfs(nodes, edges.drop('key', axis = 1), graph_attrs = graph_attrs)


In [ ]:
# Only keep mainland, irrelevant as O-D Nodes are applied.
start_node = ox.get_nearest_node(G, (59.9098, 10.7146)) 
F = G.subgraph(nx.shortest_path(G.to_undirected(), start_node)) 


In [ ]:
H = F.copy()
H = H.to_undirected()
# nx.write_gpickle(H, "Data/unsimp.gpickle") #Save sunplified 

In [ ]:
# Retrieve elevation of each node and grade of edges (You'll need your own Google Cloud Console API for this)
GOOGLE_API = "Insert API Key"
H_elev = ox.elevation.add_node_elevations_google(F, api_key= GOOGLE_API)
H_elev = ox.elevation.add_edge_grades(H_elev)

In [ ]:
H = H_elev.copy()
for i in H.edges:
    grade = H.edges[i]['grade']
    if grade == float('inf'):
        # print("grade is inf")
        H.edges[i]['grade'] = 0
    if grade == float('-inf'):
        # print("grade is -inf")
        H.edges[i]['grade'] = 0
    if np.isnan(grade):
        # print("grade is nan")
        H.edges[i]['grade'] = 0

In [ ]:
# Convert roadclass to int, because simplification appends it to list
for i in H.edges:
    data = H.edges[i]
    roadclass = data['funcroadclass']
    if type(roadclass) == list:
        # Keep minimum roadclass
        roadclass = min(roadclass)
    data['funcroadclass'] = roadclass

In [ ]:
nx.write_gpickle(H, "Data/H.gpickle") #Save unsimplified network

In [5]:
#Get nearest node to each O-D location
lat = list(OD.lat)
lon = list(OD.lon)
ODs= ox.distance.get_nearest_nodes(H, lon, lat)
OD['node'] = ODs

#Get nearest node to each CS
latA = list(CS.Latitude)
lonA = list(CS.Longitude)
AllCSnodes = ox.distance.get_nearest_nodes(H, lonA, latA)
CS['node']=AllCSnodes
AllCSnodes1 = list(CS.node)
#Create combinations of O-D pairs. Used for several functions later on
combos = combinations(ODs,2)
OD_pairs = []
for i in combos:
    OD_pairs.append(list(i))

In [ ]:
def get_edge_attributes(cumlist):
    
    
    """
    Function that takes a list of network nodes and merge their attributes as the different attributes need to be treated and formatted differently. This specifically applies for LineString.
    
    Input:
    cumlist: Cummulative list of nodes
    
    """

    cumlength = len(cumlist)
    path_attributes = dict()
    attrs_to_sum = {"length", "drivetime",}
    attrs_to_set = {'oneway', 'ref', 'name', 'funcroadclass', 'roadclass'}
    dummy_attrs = {'isFerry', 'isTunnel', 'isBridge'}
    nodes_to_remove = []
    
    for i in range(cumlength-1):
        u = cumlist[i]
        v = cumlist[i+1]
        edge_data = H.edges[u, v,0]

        for attr in edge_data:
            if attr in path_attributes:
            # if this key already exists in the dict, append it to the value list
                path_attributes[attr].append(edge_data[attr])
            else:
            # if this key doesn't already exist, set the value to a list containing the one value
                path_attributes[attr] = [edge_data[attr]]

    for attr in path_attributes:
        if attr == 'grade' or attr == 'length_weight':
            path_attributes[attr] = list(path_attributes[attr]) 
        elif attr in dummy_attrs:
            #If attribute is isFerry, isBridge og isTunnel, returns the value for first and last edge as these are candidate locations.
            if type(path_attributes[attr][0]) == list and type(path_attributes[attr][-1]) == list:
                fromnode = path_attributes[attr][0][0]
                tonode = path_attributes[attr][-1][-1]
                
            elif type(path_attributes[attr][0]) != list and type(path_attributes[attr][-1]) != list:
                fromnode = path_attributes[attr][0]
                tonode = path_attributes[attr][-1]
                
            elif type(path_attributes[attr][0]) == list and type(path_attributes[attr][-1]) != list:
                fromnode = path_attributes[attr][0][0]
                tonode = path_attributes[attr][-1]
                
            elif type(path_attributes[attr][0]) != list and type(path_attributes[attr][-1]) == list:
                fromnode = path_attributes[attr][0]
                tonode = path_attributes[attr][-1][-1]
            path_attributes[attr] = list((fromnode, tonode))

        elif attr in attrs_to_sum:
            # if this attribute must be summed, sum it now
            path_attributes[attr] = sum(path_attributes[attr])

        elif attr == 'id':
            path_attributes[attr] = list((cumlist[0],cumlist[-1]))

        
        elif attr == 'geometry':
                path_attributes[attr] = ops.linemerge(path_attributes[attr])
                
        elif attr == 'oneway':
            path_attributes[attr] = 'False'


        else:
        # otherwise, if there are multiple values, keep one of each
            try:
                path_attributes[attr] = list(set(path_attributes[attr]))
            except TypeError:
                try:
                    path_attributes[attr] = list(set(path_attributes[attr][0]))
                except:
                    path_attributes[attr] = path_attributes[attr]
            
    path_attributes['artificial'] = 1
    return path_attributes


In [ ]:
def shorten_edges_v2(G, OD, cutoff):
    
    """
    Simplifies the edges in the network in order to make the runtimes of the GA/greedy substitution feassible. Depending on the threshold value, the function will remove all nodes/edges within a certain range, store and merge
    their attributes, before creating a new edge between the first and last node in the specified range. If threshol is for example 50000, the function will remove all nodes except the first and last node for every 50th km and 
    create a new edge between the first and last node. This i then repeated for each path between each OD pair.
    
    There are also certain statements to make sure that the same nodes are used if several paths intersect. This usually happens when multiple routes shares the same path. 
    
    Input:
    G: Networkx Multigraph
    OD: CSV file with coordinates of each OD location in Norway
    Threshold: Cutoff in meters at which edges at merged. 
    """

    
    #Create flat list of O-D pairs
    OD_pairs_flat = [item for sublist in OD_pairs for item in sublist]
    
    #Create empty MultiGraph
    B = nx.MultiGraph(crs='epsg:4326')
    
    #Initialize progressbar to monitor progress of function while running.
    maxval = len(OD_pairs)
    bar    = progressbar.ProgressBar(maxval=maxval, \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

    counter=0
    bar.start()

    UsedNodes = []
    for i in OD_pairs:
        fromOD         = i[0]
        toOD           = i[1]
        path           = nx.shortest_path(H, fromOD, toOD, 'length')
        pathlength     = len(path)
        lengthofpath   = nx.shortest_path_length(H, fromOD, toOD, 'length')
        cumlength      = 0
        cumlist        = []
        cutoff         = cutoff


        #for each in edge in path[i]
        for i in range(pathlength-1):
            fromnode = path[i]
            tonode = path[i+1]
            templength = H.edges[fromnode, tonode, 0]['length']

            cumlist.append(fromnode)
            cumlist.append(tonode)
            cumlength += templength

            #First check if tonode already exists in B. If it does, it means that a previously created path has used same route. 
            #In this case we want to use existing nodes on path rather than creating new ones.
            if tonode in UsedNodes:
                #If edge does not exist, then add it
                if B.has_edge(cumlist[0], cumlist[-1]) is False:
                    cumlist = list(dict.fromkeys(cumlist))
                    #Get attributes for nodes and edge to add
                    fnodeattr = H.nodes[cumlist[0]]
                    tnodeattr = H.nodes[cumlist[-1]]
                    edgeattr = get_edge_attributes(cumlist)

                    #Add nodes and edge
                    B.add_node(cumlist[0], **fnodeattr)
                    B.add_node(cumlist[-1], **tnodeattr)
                    B.add_edge(cumlist[0], cumlist[-1], **edgeattr)

                    #Added nodes are appended to all UsedNodes
                    UsedNodes.append(cumlist[0])
                    UsedNodes.append(cumlist[-1])
                    cumlist = []
                    cumlength = 0


                #Else, skip this part, but empty cummulative path and length
                else:
                    cumlist = []
                    cumlength = 0
                    pass
                
            elif tonode in AllCSnodes: #if next node is a CS node
                #If edge does not exist, then add it
                if B.has_edge(cumlist[0], cumlist[-1]) is False:
                    cumlist = list(dict.fromkeys(cumlist))
                    #Get attributes for nodes and edge to add
                    fnodeattr = H.nodes[cumlist[0]]
                    tnodeattr = H.nodes[cumlist[-1]]
                    edgeattr = get_edge_attributes(cumlist)

                    #Add nodes and edge
                    B.add_node(cumlist[0], **fnodeattr)
                    B.add_node(cumlist[-1], **tnodeattr)
                    B.add_edge(cumlist[0], cumlist[-1], **edgeattr)

                    #Added nodes are appended to all UsedNodes
                    UsedNodes.append(cumlist[0])
                    UsedNodes.append(cumlist[-1])
                    cumlist = []
                    cumlength = 0


                #Else, skip this part, but empty cummulative path and length
                else:
                    cumlist = []
                    cumlength = 0
                    pass
                

            #Next, check if tonode is an OD node. If it is we will create edge from start of cumlist to the OD node.
            elif tonode in OD_pairs_flat:
                #If edge does not exist, then add it
                if B.has_edge(cumlist[0], cumlist[-1]) is False:
                    cumlist = list(dict.fromkeys(cumlist))
                    #Get attributes for nodes and edge to add
                    fnodeattr = H.nodes[cumlist[0]]
                    tnodeattr = H.nodes[cumlist[-1]]
                    edgeattr = get_edge_attributes(cumlist)

                    #Add nodes and edge
                    B.add_node(cumlist[0], **fnodeattr)
                    B.add_node(cumlist[-1], **tnodeattr)
                    B.add_edge(cumlist[0], cumlist[-1], **edgeattr)

                    UsedNodes.append(cumlist[0])
                    UsedNodes.append(cumlist[-1])
                    cumlist = []
                    cumlength = 0
                #Else, skip this part, but empty cummulative path and length
                else:
                    cumlist = []
                    cumlength = 0
                    pass

            #Next, check if the cummulative length in path is higher than the set threshold. If it is we will create a new edge in B
            #Can consider adding an if inside here that check if there is a node within a certain radius, and skip if it is.
            #This could prevent that nodes on completely different paths are very close or that a node is set very close to an OD node. 
            #Would then find last element in cumlist in H and get lat,lon, then use ox nearest node to find nearest node in B and finally use geopy to find distance between them. 

            elif cumlength >= threshold:            


                if B.has_edge(cumlist[0], cumlist[-1]) is False:
                    #Remove duplicates from cumlist
                    cumlist = list(dict.fromkeys(cumlist))
                    distanceOD = 100000
                    #Before we add new edge, check if destination node is within threshold/2
                    lat = H.nodes[cumlist[-1]]['y']
                    lon = H.nodes[cumlist[-1]]['x']
                    latlon = (lat,lon)
                    latdestination = H.nodes[toOD]['y']
                    londestination = H.nodes[toOD]['x']

                    #Also check if there is an already created node nearby
                    try:
                        potentialnode = ox.get_nearest_node(B, latlon)
                        clatclon = (latdestination,londestination)
                        distanceOD = (geopy.distance.geodesic(latlon,clatclon).m)

                    except ValueError:
                        pass

                    # clatclon = (latdestination,londestination)
                    # distanceOD = (geopy.distance.geodesic(latlon,clatclon).m)
                    # distancepath = geopy.distance.geodesic(latlon, platplon).m

                    if distanceOD <= threshold/2:
                        pass


                    else:
                        #Get attributes for nodes and edge to add
                        fnodeattr = H.nodes[cumlist[0]]
                        tnodeattr = H.nodes[cumlist[-1]]
                        edgeattr = get_edge_attributes(cumlist)

                        #Add nodes and edge
                        B.add_node(cumlist[0], **fnodeattr)
                        B.add_node(cumlist[-1], **tnodeattr)
                        B.add_edge(cumlist[0], cumlist[-1], **edgeattr)

                        UsedNodes.append(cumlist[0])
                        UsedNodes.append(cumlist[-1])                
                        cumlist = []
                        cumlength = 0

                #Else, skip this part, but empty cummulative path and length
                else:
                    cumlist = []
                    cumlength = 0
                    pass

        bar.update(counter)
        counter+=1

    bar.finish()

    #Set OD=1 for all O-D nodes and CS=1 for all current CS nodes. 
    nx.set_node_attributes(B, 0, 'OD')
    nx.set_node_attributes(B, 0, 'CS')

    attrs = {}
    for i in OD_pairs_flat:
        O = i
        attrs[i] = {'OD':1}
    nx.set_node_attributes(B, attrs)
    
    attrs = {}
    for i in AllCSnodes:
        O = i
        attrs[i] = {'CS':1}
    nx.set_node_attributes(B, attrs)
    
    
    print("Complete!")
    return B


In [ ]:
B = shorten_edges_v2(H, OD, 10000) #Select cut-off value
nx.write_gpickle(B,"Data/B10000.gpickle")
B = shorten_edges_v2(H, OD, 2000)
nx.write_gpickle(B,"Data/B2000.gpickle")
B = shorten_edges_v2(H, OD, 5000)
nx.write_gpickle(B,"Data/B5000.gpickle")

In [1]:
#Basic figure of network
node_color = []
for i in B.nodes:
    if B.nodes[i]['CS']==1:
            node_color.append('red')
    else:
        node_color.append('c')
ox.plot_graph(B, node_color=node_color, node_size=5)


In [54]:
#Flatten lists of lists for 'grade' and length_weight. Must be done because the simplification process creates list of lists for merged edges
#Run entire loop 10 times to ensure that all lists are flattended.
for i in range(10):
    for i in B.edges:
        flat_list_length_weight = []
        flat_list_grade = []
        if type(B.edges[i]['length_weight']) == list:
            for sublist in B.edges[i]['length_weight']:
                try:
                    for item in sublist:
                        flat_list_length_weight.append(item)
                except TypeError:
                    flat_list_length_weight.append(sublist)
            B.edges[i]['length_weight'] = flat_list_length_weight
        else:
            pass
        if type(B.edges[i]['grade']) == list:
            for sublist in B.edges[i]['grade']:
                try:
                    for item in sublist:
                        flat_list_grade.append(item)
                except TypeError:
                    flat_list_grade.append(sublist)
            B.edges[i]['grade'] = flat_list_grade
        else:
            pass

In [55]:
def calculate_eff_lengths(edge):
    
    """
    Calculates the effective length of an edge by using length weights and road gradient
    """

    startnode = edge[0]
    endnode = edge[1]
    const = 0.372
    grade_intervals = [-0.09, -0.07, -0.05, -0.03, -0.01, 0.01, 0.03, 0.05, 0.07, 0.09, 0.11]
    increased_consumption = [-0.332, -0.217, -0.148, -0.121, -0.073, 0.085, 0.152, 0.203, 0.306, 0.358, 0.552]
    
    iterator = len(B.edges[edge]['length_weight'])
    
    #If the direction of the edge is the default fromnode -> tonode, we set a multiplier to 1, meaning that the edge gradients will be default:
    if B.nodes[startnode]['elevation'] < B.nodes[endnode]['elevation']: 
        direction = 1
    #If direction is reversed, meaning that the vehicle is returning to the origin node, we reverse the gradients
    else:
        direction = -1
    eff_length_trueway = 0
        
    for i in range(iterator):
        edge_grad = B.edges[edge]['grade'][i]*(direction)
        length_weight = B.edges[edge]['length_weight'][i]

        if edge_grad < grade_intervals[0]: #less than -9%
            diff = const + increased_consumption[0]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[0] < edge_grad <= grade_intervals[1]: #-9% to -7%
            diff = const + increased_consumption[1]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[1] < edge_grad <= grade_intervals[2]: #-7% to -5%
            diff = const + increased_consumption[2]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[2] < edge_grad <= grade_intervals[3]: #-5% to -3%
            diff = const + increased_consumption[3]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[3] <= edge_grad <= grade_intervals[4]: #-3% to -1%
            diff = const + increased_consumption[4]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[5] <= edge_grad <= grade_intervals[6]: #1% to #3%
            diff = const + increased_consumption[5]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[6] < edge_grad <= grade_intervals[7]: #3% to 5%
            diff = const + increased_consumption[6]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[7] < edge_grad <= grade_intervals[8]: #5% to 7%
            diff = const + increased_consumption[7]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[8] < edge_grad <= grade_intervals[9]: #7% to 9%
            diff = const + increased_consumption[8]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[9] < edge_grad <= grade_intervals[10]: #9% to 11%
            diff = const + increased_consumption[9]
            eff_length_trueway += length_weight*(diff/const)

        elif edge_grad > grade_intervals[10]: #more then 11%
            diff = const + increased_consumption[0]
            eff_length_trueway += length_weight*(diff/const)


        else: #If grade is between -1% and 1%
            eff_length_trueway += length_weight
            diff = 0
     
    return eff_length_trueway

In [56]:
#Create df of all effective lengths for both directions
#Also create df containing total range between OD pairs both directions
alledges = []
true = []
for i in B.edges:
    startnode = i[0]
    endnode = i[1]    
    trueway = [startnode,endnode,0]
    alledges.append(trueway)
    true.append("true")
    revway = [endnode, startnode,0]
    alledges.append(revway)
    true.append("false")
    
edge = []
eff_len = []
lengths = pd.DataFrame(columns = ['fromnode','tonode', 'eff_len'])
for i in alledges:
    startnode = i[0]
    endnode = i[1]
    bothnodes = startnode,endnode
    
    edge.append(list(i))
    result = calculate_eff_lengths(list(i))
    eff_len.append(result)
    
    if list(bothnodes) == B.edges[i]['id']:
        B.edges[i]['eff_len_t'] = result
    else:
        B.edges[i]['eff_len_f'] = result

In [58]:
#Create df

#First add w and w_s to each node
for count, value in enumerate(OD['node']):
    B.nodes[value]['w']   = OD.iloc[count]['w']
    B.nodes[value]['w_s'] = OD.iloc[count]['w_s']
    
#Next, create df
df = pd.DataFrame(columns = ['fromnode', 'tonode', 'drivetime', 'length', 'weight', 'weight_s', 'path'])

for i in OD_pairs:
    tempfrom, tempto = i
    
    path       = nx.shortest_path(B, tempfrom, tempto, 'length')
    pathlength = nx.shortest_path_length(B, tempfrom, tempto, 'length')
    pathlen    = len(path)
    weight     = float(OD.loc[OD['node']==tempfrom]['w'])*float(OD.loc[OD['node']==tempto]['w'])
    weight_s   = float(OD.loc[OD['node']==tempfrom]['w_s'])*float(OD.loc[OD['node']==tempto]['w_s'])
    drivetime  = 0
    
    for i in range(pathlen-1):
        drivetime += B.edges[path[i],path[i+1],0]['drivetime']
    
    temp = tempfrom, tempto, drivetime, pathlength, weight, weight_s, path
    seriestoappend = pd.Series(temp, index=df.columns)
    df = df.append(seriestoappend, ignore_index=True)    


In [59]:
#Next, remove faulty added nodes and edges. These would only act as noise when visualising.

for i in B.edges:
    B.edges[i]['used'] =0

for i in B.nodes:
    B.nodes[i]['used'] = 0
    
for i in df.index:
    path = df.iloc[i]['path']
    for p in range(len(path)):
        try:
            a = path[p]
            b = path[p+1]
            B.edges[a,b,0]['used'] = 1
            B.nodes[a]['used']=1
            B.nodes[b]['used']=1
        except:
            pass    
unusedNodes = [] #List of unused nodes
for i in B.nodes:
    if B.nodes[i]['used'] == 0:
        unusedNodes.append(i)   
        
unusedEdges = []
for i in B.edges:
    if B.edges[i]['used'] ==0:
        unusedEdges.append(i)

#Remove all unused nodes
for n in unusedNodes:
    B.remove_node(n)

#Remove all unused edges
for e in unusedEdges:
    a = e[0]
    b = e[1]
    try:
        B.remove_edge(a,b)
    except:
        pass

In [60]:
#Create dictionary of effective lengths for each OD pair in both directions. Store lengths in dictionary because this is more time efficient.
keys = []
vals = []
for i in range(len(df)):
    fromnode = df.iloc[i]['fromnode']
    tonode = df.iloc[i]['tonode']
    path = df.iloc[i]['path']
    eff_len_default =0
    eff_len_false = 0
    for i in range(len(path)):
        try:
            edge = [path[i],path[i+1],0]
            eff_edge_len = calculate_eff_lengths(edge)
            eff_len_default += eff_edge_len
        except IndexError:
            pass
    path.reverse()
    for i in range(len(path)):
        try:
            eff_edge_len = calculate_eff_lengths(edge)
            eff_len_false += eff_edge_len
        except IndexError:
            pass
    fromto = fromnode, tonode
    tofrom = tonode,fromnode
    keys.append(fromto)
    keys.append(tofrom)
    vals.append(eff_len_default)
    vals.append(eff_len_false)
zip_iter = zip(keys,vals)
lengths_dict = dict(zip_iter)

In [61]:
#Define gravity model that estimates the flow between each O-D pair
def GravityModel(master_df):
    flow=[]
    master_df['flow']=0
    for i in master_df.index:
        GC=(0.75*(master_df['length'][i])+0.5*master_df['drivetime'][i])
        flow.append(master_df['weight'][i]/GC)
    
    tot_flow=sum(flow)
    for i in master_df.index:
        master_df['flow'][i]=flow[i]/tot_flow
GravityModel(df)

In [ ]:
#Add rest of CS stations that can be localized within a 10km distance.
bnodes = []
name = []
Latitude = []
Longitude = []
node = []
for count, i in enumerate(list(AllCSnodes)):
    # if i not in B.nodes:
    try:
        latCS = float(CS[CS['node']==i]['Latitude'])
        lonCS = float(CS[CS['node']==i]['Longitude'])
        tempname = CS[CS['node']==i]['Name'][count]
    except TypeError:
        latCS = list(CS[CS['node']==i]['Latitude'])[0]
        lonCS = list(CS[CS['node']==i]['Longitude'])[0]
        tempname = CS[CS['node']==i]['Name'][count]
    potentialnode = ox.nearest_nodes(B, lonCS, latCS)
    lat = B.nodes[potentialnode]['y']
    lon = B.nodes[potentialnode]['x']
    dist = geopy.distance.geodesic((lat,lon),(latCS, lonCS)).m
    if dist <10000 and B.nodes[potentialnode]['CS']==0:
        name.append(tempname)
        Latitude.append(latCS)
        Longitude.append(lonCS)
        node.append(potentialnode)
        B.nodes[potentialnode]['CS']=1 
d = {'Name':name, 'Latitude':Latitude, 'Longitude':Longitude, 'node':node}
CS_simplified = pd.DataFrame(d)

In [ ]:
#Finally export all items
nx.write_gpickle(B, "Data\B10000-CS2.gpickle")
nx.write_gpickle(H, "Data\H.gpickle")
df.to_pickle("Data/flows.csv")
df.to_pickle("Data/CS_simplified.csv")
with open('Data/lengths.pickle', 'wb') as handle:
    pickle.dump(lengths_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Data/AllCSnodes.pickle', 'wb') as handle:
    pickle.dump(AllCSnodes, handle, protocol=pickle.HIGHEST_PROTOCOL)
